#  Prepare (Import Libraries, etc)

In [2]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.components.trainer.fn_args_utils import FnArgs
from tfx.proto import example_gen_pb2 as exgen, trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
import os

In [3]:
name = "detector-pipeline"
schema = "detector-tfdv-schema"

root = os.path.join('pipelines', name)

metadata = os.path.join('metadata', name, 'metadata.db')

serving_model = os.path.join('serving_model', name)

In [4]:
data = "data"

In [5]:
inter = InteractiveContext(pipeline_root=root)

# Example GEN

In [6]:
output = exgen.Output(
    split_config = exgen.SplitConfig(splits=[
         exgen.SplitConfig.Split(name="train", hash_buckets=8),
         exgen.SplitConfig.Split(name='eval', hash_buckets=2)
    ])
)

In [7]:
ex_gen = CsvExampleGen(input_base = data, output_config = output)

In [8]:
inter.run(ex_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Statistics GEN

In [9]:
stat = StatisticsGen(
    examples = ex_gen.outputs['examples']
)

In [10]:
#  running
inter.run(stat)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [11]:
#  showing
inter.show(stat.outputs['statistics'])

#  Schema GEN

In [12]:
sch =  SchemaGen(
    statistics = stat.outputs['statistics']
)

In [13]:
#  running
inter.run(sch)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [14]:
#  showing
inter.show(sch.outputs['schema'])

Type  Presence Valency Domain
Feature name                                
'Label'         INT  required              -
'Text'        BYTES  required              -

# Example Validator

In [15]:
exval = ExampleValidator(
    statistics = stat.outputs['statistics'],
    schema = sch.outputs['schema']
)

In [16]:
inter.run(exval)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [17]:
inter.show(exval.outputs['anomalies'])

#  Transform

In [18]:
trans_file = 'detector_transform.py'

In [19]:
%%writefile {trans_file}

import tensorflow as tf

FEATURE_KEY = "Text"
LABEL_KEY = 'Label'

def transformed(key):
  return key + "_xf"

def preprocessing_fn(input):
  output = {}

  output[transformed(FEATURE_KEY)] = tf.strings.lower(input[FEATURE_KEY])
  output[transformed(LABEL_KEY)] = tf.cast(input[LABEL_KEY], tf.int64)

  return output

Writing detector_transform.py


In [20]:
transform = Transform(
    examples = ex_gen.outputs['examples'],
    schema = sch.outputs['schema'],
    module_file = os.path.abspath(trans_file)
)

In [21]:
inter.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Trainer

In [22]:
trainer_file = "detector_trainer.py"

In [23]:
%%writefile {trainer_file}

import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_transform as tft
from keras.layers import *
from keras.metrics import *
from keras.callbacks import *
from keras.optimizers import *
from tfx.components.trainer.fn_args_utils import FnArgs

FEATURE_KEY = "Text"
LABEL_KEY = 'Label'
NUM_EPOCHS = 10

def transformed_name(key):
  return key + "_xf"

def gzip_reader_fn(filenames):
  return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, epochs, baze = 64) -> tf.data.Dataset:

  transform_feature_spec = (tf_transform_output.transformed_feature_spec().copy())

  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern = file_pattern,
      batch_size = baze,
      features = transform_feature_spec,
      reader = gzip_reader_fn,
      num_epochs = epochs,
      label_key = transformed_name(LABEL_KEY)
  )
  return dataset

vocab = 5000
length = 19

vect = TextVectorization(
    max_tokens = vocab,
    output_mode = 'int',
    output_sequence_length = length
)

emb_dim = 16

def modeling():
    input = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    text_vectorized = vect(input)

    embd = Embedding(vocab, emb_dim, input_length=length, name='embedding')(text_vectorized)
    lstm1 = Bidirectional(LSTM(32))(embd)

    dense1 = Dense(16, 'relu')(lstm1)
    output = Dense(1, 'sigmoid')(dense1)

    model = tf.keras.Model(inputs=input, outputs=output)

    model.compile(
        optimizer = Adam(0.001),
        loss = 'binary_crossentropy',
        metrics = ['binary_accuracy']
    )

    model.summary()
    return model

def _get_serve_tf_examples_fn(model, tf_transform_output):

  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function
  def serve_tf_examples_fn(serialized_ex):

    feature_spec = tf_transform_output.raw_feature_spec()

    feature_spec.pop(LABEL_KEY)

    parsed = tf.io.parse_example(serialized_ex, feature_spec)

    transformed_features = model.tft_layer(parsed)

    return model(transformed_features)

  return serve_tf_examples_fn

def run_fn(fn_args: FnArgs) -> None:

  log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

  tenbor = TensorBoard(
      log_dir = log_dir,
      update_freq = 'batch'
  )

  es = EarlyStopping(
      monitor = 'val_binary_accuracy',
      mode = 'max',
      patience = 2,
      verbose = 1
  )

  mc = ModelCheckpoint(
      fn_args.serving_model_dir,
      mode = 'max',
      monitor = 'val_binary_accuracy',
      save_best_only = True
  )

  tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

  train_set = input_fn(fn_args.train_files, tf_transform_output, NUM_EPOCHS)
  val_set = input_fn(fn_args.eval_files, tf_transform_output, NUM_EPOCHS)
  text_data = [j[0].numpy()[0] for j in [i[0][transformed_name(FEATURE_KEY)] for i in list(train_set)]]
  vect.adapt(text_data)

  model = modeling()

  model.fit(x = train_set,
            steps_per_epoch=fn_args.train_steps,
            validation_data = val_set,
            callbacks = [tenbor, es, mc],
            validation_steps = fn_args.eval_steps,
            epochs = NUM_EPOCHS)

  signature = {
      'serving_default' :
      _get_serve_tf_examples_fn(
          model,
          tf_transform_output
      ).get_concrete_function(tf.TensorSpec(shape = [None],
                                            dtype = tf.string,
                                            name = 'examples'))
  }
  model.save(fn_args.serving_model_dir, save_format = 'tf', signatures = signature)

Writing detector_trainer.py


In [24]:
train = Trainer(
    module_file = os.path.abspath(trainer_file),
    examples = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema = sch.outputs['schema'],
    train_args = trainer_pb2.TrainArgs(splits=['train']),
    eval_args = trainer_pb2.EvalArgs(splits=['eval'])
)

In [25]:
inter.run(train)

Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Text_xf (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 19)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 19, 16)            80000     
                                                                 
 bidirectional (Bidirection  (None, 64)                12544     
 al)                                                             
                                                                 
 dense (Dense)               (None, 16)                1040      
                                                                 
 dense_1 (Dense)             (None, 1)                 17    

ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

#  Resolver

In [26]:
model_resolver = Resolver(
    strategy_class = LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Lates_blessed_model_resolver')

In [27]:
inter.run(model_resolver)

ExecutionResult(
    component_id: Lates_blessed_model_resolver
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Lates_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Lates_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

#  Evaluator

In [28]:
import tensorflow_model_analysis as tfma

eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='Label')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[

            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
)

In [29]:
evaluator = Evaluator(
    examples=ex_gen.outputs['examples'],
    model=train.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)


In [30]:
inter.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [31]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

#  Pusher

In [32]:
pusher = Pusher(
    model = train.outputs['model'],
    model_blessing = evaluator.outputs['blessing'],
    push_destination = pusher_pb2.PushDestination(
        filesystem = pusher_pb2.PushDestination.Filesystem(
            base_directory = 'serving_model_dir/ai-vs-human-text'
        )
    )
)

In [33]:
inter.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 9
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)